In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install fast-plate-ocr[onnx]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.3 MB/s eta 0:00:00


In [34]:
import torch

if torch.cuda.is_available():
    print("GPU attiva:", torch.cuda.get_device_name(0))
else:
    print("GPU non disponibile")


GPU non disponibile


In [6]:
!pip install -r /content/drive/MyDrive/IARTS/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.2 MB/s eta 0:00:00
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=32078badc0d66ed66fc5519d32130890ab6ac26e017138f30349a03ed58d58ae
  Stored in directory: /root/.cache/pip/wheels/e7/5d/0e/4b4fff9a47468fed5633211fb3b76d1db43fe806a17fb7486a
  Created wheel for grad-cam: filename=grad_cam-1.5.5-py3-none-any.whl size=44284 sha256=f8470b2456481581f5225adb96f835582f3a0f5409525794b3d7bb32cdad4b56
  Stored in directory: /root/.cache/pip/wheels/fb/3b/09/2afc520f3d69bc26ae6bd87416759c820a3f7d05c1a077bbf6
Successfully built lime grad-cam


In [7]:
import os, glob, zipfile
import cv2, numpy as np, os
import xml.etree.ElementTree as ET
import shutil, json, random
from ultralytics import YOLO
import torch

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
'''
zip_path = "/content/drive/MyDrive/IARTS/dataset/archive.zip"
extract_path = "/content/drive/MyDrive/IARTS/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
'''

In [39]:
#SE NON CI STANNO LE CREA, ALTRIMENTI DEFINISCE SOLO LE VARIABILI

BASE = "/content/drive/MyDrive/IARTS/dataset"
RAW_IMG_DIR = f"{BASE}/images"
RAW_ANN_DIR = f"{BASE}/annotations"
PROC_DIR    = f"{BASE}/processed"       # immagini preprocessate (FFT)            # labels YOLO + splits
CROPS_DIR   = f"{BASE}/crops"           # targhe ritagliate
RESULTS_DIR = f"/content/drive/MyDrive/IARTS/results"
IMG_4_YOLO  = f"{BASE}/images_yolo"
IMG_4_YOLO_ORIGINAL  = f"{BASE}/images_yolo_original"

for d in [PROC_DIR, CROPS_DIR, RESULTS_DIR, IMG_4_YOLO, IMG_4_YOLO_ORIGINAL]:
    os.makedirs(d, exist_ok=True)


In [ ]:
def find_subdir(root, candidate_names):
    for name in candidate_names:
        p = os.path.join(root, name)
        if os.path.isdir(p):
            return p
    return None

RAW_IMG_DIR = find_subdir(BASE, ["images", "JPEGImages", "img", "imgs"]) or RAW_IMG_DIR
RAW_ANN_DIR = find_subdir(BASE, ["annotations", "Annotations", "xml"]) or RAW_ANN_DIR

print("Immagini:", RAW_IMG_DIR)
print("Annotazioni:", RAW_ANN_DIR)


Immagini: /content/drive/MyDrive/IARTS/dataset/images
Annotazioni: /content/drive/MyDrive/IARTS/dataset/annotations


In [ ]:
#TRASFORMATA FOURIER (E' STATO FATTO SIA PASSALTO CHE PASSABASSO) E LE IMMAGINI
#SONO POI STATE SALVATE DENTRO IARTS/dataset/processed/

MODES    = ["low", "high"]      # Passabasso e passalto
CUTOFFS  = [20, 30, 40]         # raggi filtro(20=stretto, quindi passa-basso super sfocato / passa-alto molto aggressivo)

def fourier_filter_color(img_bgr, mode="low", cutoff=30):
    ycrcb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2YCrCb)
    Y, Cr, Cb = cv2.split(ycrcb)

    f = np.fft.fft2(Y)
    fshift = np.fft.fftshift(f)

    rows, cols = Y.shape
    crow, ccol = rows//2, cols//2
    mask = np.zeros((rows, cols), np.uint8)

    if mode == "low":
        cv2.circle(mask, (ccol, crow), cutoff, 1, -1)
    elif mode == "high":
        mask[:] = 1
        cv2.circle(mask, (ccol, crow), cutoff, 0, -1)
    else:
        raise ValueError("mode deve essere 'low' o 'high'")

    fshift_filtered = fshift * mask
    f_ishift = np.fft.ifftshift(fshift_filtered)
    Y_filt = np.fft.ifft2(f_ishift)
    Y_filt = np.abs(Y_filt).clip(0,255).astype(np.uint8)

    out = cv2.merge([Y_filt, Cr, Cb])
    return cv2.cvtColor(out, cv2.COLOR_YCrCb2BGR)

img_paths = sorted(glob.glob(os.path.join(RAW_IMG_DIR, "*.*")))
assert len(img_paths) > 0, f"Nessuna immagine trovata in {RAW_IMG_DIR}"

processed_sets = []

for mode in MODES:
    for cutoff in CUTOFFS:
        outdir = f"{BASE}/processed/processed_{mode}_{cutoff}"
        os.makedirs(outdir, exist_ok=True)
        print(f"\n>> Processing {mode} (cutoff={cutoff}) → {outdir}")
        count = 0
        for p in img_paths:
            img = cv2.imread(p)
            if img is None:
                continue
            proc = fourier_filter_color(img, mode=mode, cutoff=cutoff)
            cv2.imwrite(os.path.join(outdir, os.path.basename(p)), proc)
            count += 1
            if count % 100 == 0:
                print(f"  - {count} immagini...")
        print(f" Finito: {count} immagini salvate.")
        processed_sets.append(outdir)

print("\nSet preprocessati disponibili:")
for s in processed_sets:
    print(" -", s)




>> Processing low (cutoff=20) → /content/drive/MyDrive/IARTS/dataset/processed/processed_low_20
  - 100 immagini...
 Finito: 136 immagini salvate.

>> Processing low (cutoff=30) → /content/drive/MyDrive/IARTS/dataset/processed/processed_low_30
  - 100 immagini...
 Finito: 136 immagini salvate.

>> Processing low (cutoff=40) → /content/drive/MyDrive/IARTS/dataset/processed/processed_low_40
  - 100 immagini...
 Finito: 136 immagini salvate.

>> Processing high (cutoff=20) → /content/drive/MyDrive/IARTS/dataset/processed/processed_high_20
  - 100 immagini...
 Finito: 136 immagini salvate.

>> Processing high (cutoff=30) → /content/drive/MyDrive/IARTS/dataset/processed/processed_high_30
  - 100 immagini...
 Finito: 136 immagini salvate.

>> Processing high (cutoff=40) → /content/drive/MyDrive/IARTS/dataset/processed/processed_high_40
  - 100 immagini...
 Finito: 136 immagini salvate.

Set preprocessati disponibili:
 - /content/drive/MyDrive/IARTS/dataset/processed/processed_low_20
 - /con

In [ ]:
import cv2
import numpy as np
import os
import glob

# --- Define Fuzzy Logic Preprocessing Functions ---

# Definition of the trapezoidal fuzzy membership function
def trapezoidal_membership(x, a=0.1, b=0.2, c=0.6, d=0.8):
    if x < a or x > d: return 0
    elif a <= x < b: return (x - a) / (b - a)
    elif b <= x <= c: return 1
    elif c < x <= d: return (d - x) / (d - c)

# Function to apply fuzzy edge detection to a color image
def fuzzy_filter_color(img_bgr, params):
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    sobelx = cv2.Sobel(img_gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(img_gray, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(sobelx**2 + sobely**2)
    gradient_norm = cv2.normalize(gradient_magnitude, None, 0, 1, cv2.NORM_MINMAX)

    fuzzy_edges = np.vectorize(lambda x: trapezoidal_membership(x, **params))(gradient_norm)
    fuzzy_edges_scaled = (fuzzy_edges * 255).clip(0, 255).astype(np.uint8)
    return cv2.cvtColor(fuzzy_edges_scaled, cv2.COLOR_GRAY2BGR)


# --- Main Processing Loop for Fuzzy Logic ONLY ---

img_paths = sorted(glob.glob(os.path.join(RAW_IMG_DIR, "*.*")))
assert len(img_paths) > 0, f"Nessuna immagine trovata in {RAW_IMG_DIR}"

# Find existing Fourier-processed sets and initialize the list with them
# This ensures we don't lose the work already done
processed_sets = sorted([p for p in glob.glob(f"{PROC_DIR}/processed_*") if os.path.isdir(p) and 'fuzzy' not in os.path.basename(p)])
print("Set Fourier già esistenti trovati:")
for s in processed_sets:
    print(" -", s)

# Define Fuzzy Logic parameters to test
FUZZY_PARAMS = [
    {'a': 0.1, 'b': 0.2, 'c': 0.6, 'd': 0.8},
    {'a': 0.2, 'b': 0.4, 'c': 0.7, 'd': 0.9}
]

# Process with Fuzzy Logic Filters and append to the list
for params in FUZZY_PARAMS:
    param_str = f"fuzzy_a{params['a']}_b{params['b']}_c{params['c']}_d{params['d']}"
    outdir = f"{PROC_DIR}/processed_{param_str}"
    os.makedirs(outdir, exist_ok=True)
    print(f"\n>> Creazione set con Logica Fuzzy (params={param_str}) → {outdir}")

    # Check if images already exist to avoid reprocessing
    if len(glob.glob(os.path.join(outdir, "*.png"))) == len(img_paths) or len(glob.glob(os.path.join(outdir, "*.jpg"))) == len(img_paths):
        print(f" Immagini già presenti in {outdir}. Salto la creazione.")
    else:
        count = 0
        for p in img_paths:
            img = cv2.imread(p)
            if img is None: continue
            proc = fuzzy_filter_color(img, params)
            cv2.imwrite(os.path.join(outdir, os.path.basename(p)), proc)
            count += 1
        print(f" Finito: {count} immagini salvate.")

    # Add the new directory to the list if it's not already there
    if outdir not in processed_sets:
        processed_sets.append(outdir)

print("\n✅ Preprocessing completato. Lista aggiornata dei set disponibili:")
for s in sorted(processed_sets):
    print(" -", s)

Set Fourier già esistenti trovati:
 - /content/drive/MyDrive/IARTS/dataset/processed/processed_high_20
 - /content/drive/MyDrive/IARTS/dataset/processed/processed_high_30
 - /content/drive/MyDrive/IARTS/dataset/processed/processed_high_40
 - /content/drive/MyDrive/IARTS/dataset/processed/processed_low_20
 - /content/drive/MyDrive/IARTS/dataset/processed/processed_low_30
 - /content/drive/MyDrive/IARTS/dataset/processed/processed_low_40

>> Creazione set con Logica Fuzzy (params=fuzzy_a0.1_b0.2_c0.6_d0.8) → /content/drive/MyDrive/IARTS/dataset/processed/processed_fuzzy_a0.1_b0.2_c0.6_d0.8
 Finito: 136 immagini salvate.

>> Creazione set con Logica Fuzzy (params=fuzzy_a0.2_b0.4_c0.7_d0.9) → /content/drive/MyDrive/IARTS/dataset/processed/processed_fuzzy_a0.2_b0.4_c0.7_d0.9
 Finito: 136 immagini salvate.

✅ Preprocessing completato. Lista aggiornata dei set disponibili:
 - /content/drive/MyDrive/IARTS/dataset/processed/processed_fuzzy_a0.1_b0.2_c0.6_d0.8
 - /content/drive/MyDrive/IARTS/dat

In [ ]:
#SUDDIVISIONE DEL DATASET IN TEST,VAL E TRAIN...LA SUDDIVISIONE STA IN IARTS/dataset/fixed_split.json

IM_DIR_REF = f"{BASE}/processed/processed_high_30"  # uno qualunque dei set
all_imgs = sorted([p for ext in ("*.jpg","*.png","*.jpeg")
                   for p in glob.glob(os.path.join(IM_DIR_REF, ext))])
random.seed(42)  # fisso
random.shuffle(all_imgs)

n = len(all_imgs)
n_test = int(0.15*n)
n_val  = int(0.15*(n - n_test))
splits = {
    "train": sorted(all_imgs[n_test+n_val:]),
    "val":   sorted(all_imgs[n_test:n_test+n_val]),
    "test":  sorted(all_imgs[:n_test])
}
with open(f"{BASE}/fixed_split.json","w") as f:
    json.dump({k:[os.path.basename(x) for x in v] for k,v in splits.items()}, f, indent=2)

print("✅ Split salvato in fixed_split.json")


✅ Split salvato in fixed_split.json


In [40]:
#CONVERSIONE DA VOC A YOLO, VOC(xml) -> YOLO(txt)
#Per ogni immagine abbiamo :
#Copiato l’immagine nella cartella corretta (images/train, images/val, images/test).
#Letto il .xml, estratto le box, convertito in formato YOLO, e scritto il .txt in labels/train, ecc
#IL RISULTATO SI TROVA DENTRO IMAGES_YOLO


with open(f"{BASE}/fixed_split.json") as f:
    split_names = json.load(f)

RAW_ANN_DIR = f"{BASE}/annotations"
CLASS_ID = 0

def parse_boxes(xml_file):
    t = ET.parse(xml_file); r = t.getroot()
    w = int(r.find('size/width').text); h = int(r.find('size/height').text)
    boxes=[]
    for obj in r.findall('object'):
        b = obj.find('bndbox')
        xmin,ymin,xmax,ymax = map(lambda x:int(float(x.text)), [b.find('xmin'),b.find('ymin'),b.find('xmax'),b.find('ymax')])
        boxes.append(((w,h),(xmin,ymin,xmax,ymax)))
    return boxes

def voc2yolo_bbox(w,h,xmin,ymin,xmax,ymax):
    x=(xmin+xmax)/2.0/w; y=(ymin+ymax)/2.0/h
    bw=(xmax-xmin)/w; bh=(ymax-ymin)/h
    return x,y,bw,bh

def build_yolo_set(IM_DIR, outdir):
    yimg=f"{outdir}/images"; ylbl=f"{outdir}/labels"
    for s in ["train","val","test"]:
        os.makedirs(f"{yimg}/{s}", exist_ok=True)
        os.makedirs(f"{ylbl}/{s}", exist_ok=True)
        for name in split_names[s]:
            src = os.path.join(IM_DIR, name)
            if not os.path.exists(src): continue
            shutil.copy2(src, f"{yimg}/{s}/{name}")
            xmlp = os.path.join(RAW_ANN_DIR, os.path.splitext(name)[0]+".xml")
            if not os.path.exists(xmlp): continue
            boxes = parse_boxes(xmlp)
            with open(f"{ylbl}/{s}/{os.path.splitext(name)[0]}.txt","w") as f:
                for (w,h),(xmin,ymin,xmax,ymax) in boxes:
                    x,y,bw,bh = voc2yolo_bbox(w,h,xmin,ymin,xmax,ymax)
                    f.write(f"{CLASS_ID} {x:.6f} {y:.6f} {bw:.6f} {bh:.6f}\n")
    with open(f"{outdir}/data.yaml","w") as f:
        f.write(f"path: {outdir}\ntrain: images/train\nval: images/val\ntest: images/test\nnames:\n  0: license-plate\n")

# Esegui per tutti i set processed_*
sets = sorted([p for p in glob.glob(f"{BASE}/images/") if os.path.isdir(p)])
YOLO_ROOT = f"{IMG_4_YOLO_ORIGINAL}"
yolo_dirs = []
for s in sets:
    tag = os.path.basename(s).replace("original_","yolo_")
    outdir = f"{YOLO_ROOT}/{tag}"
    build_yolo_set(s, outdir)
    yolo_dirs.append(outdir)
    print("✅ YOLO set creato:", outdir)

'''
# Esegui per tutti i set processed_*
sets = sorted([p for p in glob.glob(f"{BASE}/processed/processed_*") if os.path.isdir(p)])
YOLO_ROOT = f"{IMG_4_YOLO}"
yolo_dirs = []
for s in sets:
    tag = os.path.basename(s).replace("processed_","yolo_")
    outdir = f"{YOLO_ROOT}/{tag}"
    build_yolo_set(s, outdir)
    yolo_dirs.append(outdir)
    print("✅ YOLO set creato:", outdir)
'''

✅ YOLO set creato: /content/drive/MyDrive/IARTS/dataset/images_yolo_original/


'\n# Esegui per tutti i set processed_*\nsets = sorted([p for p in glob.glob(f"{BASE}/processed/processed_*") if os.path.isdir(p)])\nYOLO_ROOT = f"{IMG_4_YOLO}"\nyolo_dirs = []\nfor s in sets:\n    tag = os.path.basename(s).replace("processed_","yolo_")\n    outdir = f"{YOLO_ROOT}/{tag}"\n    build_yolo_set(s, outdir)\n    yolo_dirs.append(outdir)\n    print("✅ YOLO set creato:", outdir)\n'

In [42]:
#ADDESTRATO, VALUTATO E INSERITI RISULTATI DENTRO IARTS/dataset/results/metrics_yolo/yolo_metrics.json

PROJECT_DIR = f"{RESULTS_DIR}/yolo_train"
os.makedirs(PROJECT_DIR, exist_ok=True)

metrics_summary = []
for ydir in yolo_dirs:
    tag = os.path.basename(ydir)
    model = YOLO("yolov8n.pt")
    res = model.train(data=f"{ydir}/data.yaml", imgsz=640, epochs=50, batch=16,
                      device=0 if torch.cuda.is_available() else "cpu",
                      project=PROJECT_DIR, name=tag)
    m = model.val(data=f"{ydir}/data.yaml")
    metrics_summary.append({
        "set": tag,
        "map50": float(m.box.map50),
        "precision": float(m.box.mp),
        "recall": float(m.box.mr)
    })

with open(f"{RESULTS_DIR}/yolo_metrics_original.json","w") as f:
    json.dump(metrics_summary, f, indent=2)

print("Metriche YOLO salvate in:", f"{RESULTS_DIR}/metrics_yolo/yolo_metrics_original.json")



Ultralytics 8.3.193 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (AMD EPYC 7B12)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/IARTS/dataset/images_yolo_original//data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, p

In [44]:
#PREPARO IL MATERIALE PER LA VALUTAZIONE
best_tag = "yolo_low_30"  # cambialo a seconda di quello che vuoi utilizzare

# pesi migliori
best_weights = f"{RESULTS_DIR}/yolo_train/{best_tag}/weights/best.pt"
assert os.path.exists(best_weights), f"Non trovo i pesi: {best_weights}"

# cartella test
TEST_IMG_DIR = f"{BASE}/images_yolo/{best_tag}/images/test"
assert os.path.isdir(TEST_IMG_DIR), f"Non trovo la cartella test: {TEST_IMG_DIR}"

print("Userò i pesi:", best_weights)
print("Userò test images in:", TEST_IMG_DIR)


Userò i pesi: /content/drive/MyDrive/IARTS/results/yolo_train/train2/weights/best.pt
Userò test images in: /content/drive/MyDrive/IARTS/dataset/images_yolo_original/images/test


In [45]:
#ESEGUO IL MODELLO SUL TEST SET E RIESCE COME RICHIESTO A INQUADRARE ESATTAMENTE TUTTE LE TARGHE

# Carica i pesi migliori (best.pt) dentro l'oggetto modello
model = YOLO(best_weights)

PROJECT_DIR = f"{RESULTS_DIR}/yolo_pred"
os.makedirs(PROJECT_DIR, exist_ok=True)

# Lancia l'inferenza sulla cartella di test
pred = model.predict(
    source=TEST_IMG_DIR,    # cartella con le immagini di test (può essere anche un file singolo, URL, video)
    imgsz=640,              # dimensione di input (ridimensiona l’immagine per la rete)
    conf=0.25,              # confidence threshold: filtra predizioni sotto 0.25
    device=0 if torch.cuda.is_available() else "cpu",  # usa GPU se c'è, altrimenti CPU
    save=True,              # salva le immagini con le bbox disegnate
    project=PROJECT_DIR,    # directory base dove salvare gli output
    name=f"pred_{best_tag}" # sottocartella dell’output (es: results/pred_yolo_low_30)
)

print("✅ Inferenza completata. Output in:", f"{RESULTS_DIR}/pred_{best_tag}")



image 1/20 /content/drive/MyDrive/IARTS/dataset/images_yolo_original/images/test/Cars367.png: 448x640 1 license-plate, 117.7ms
image 2/20 /content/drive/MyDrive/IARTS/dataset/images_yolo_original/images/test/Cars374.png: 352x640 1 license-plate, 98.2ms
image 3/20 /content/drive/MyDrive/IARTS/dataset/images_yolo_original/images/test/Cars380.png: 384x640 1 license-plate, 103.8ms
image 4/20 /content/drive/MyDrive/IARTS/dataset/images_yolo_original/images/test/Cars382.png: 480x640 1 license-plate, 124.3ms
image 5/20 /content/drive/MyDrive/IARTS/dataset/images_yolo_original/images/test/Cars387.png: 448x640 1 license-plate, 115.5ms
image 6/20 /content/drive/MyDrive/IARTS/dataset/images_yolo_original/images/test/Cars394.png: 480x640 (no detections), 116.2ms
image 7/20 /content/drive/MyDrive/IARTS/dataset/images_yolo_original/images/test/Cars410.png: 640x512 1 license-plate, 142.6ms
image 8/20 /content/drive/MyDrive/IARTS/dataset/images_yolo_original/images/test/Cars422.png: 384x640 (no detec

In [46]:
#SU QUESTE IMMAGINI USATE, TAGLIO LA PARTE SOLO DOVE CI STANNO LE TARGHE E SALVO TUTTO NELLA CARTELLA CROPS

CROPS = f"{CROPS_DIR}/crops_{best_tag}"
os.makedirs(CROPS, exist_ok=True)

for r in pred:
    im = r.orig_img
    base = os.path.splitext(os.path.basename(r.path))[0]
    if r.boxes is None:
        continue
    for i, box in enumerate(r.boxes.xyxy.cpu().numpy()):
        x1,y1,x2,y2 = map(int, box[:4])
        crop = im[max(0,y1):max(0,y2), max(0,x1):max(0,x2)]
        if crop.size == 0:
            continue
        cv2.imwrite(os.path.join(CROPS, f"{base}_{i}.png"), crop)
print("✅ Crops salvati in:", CROPS)

✅ Crops salvati in: /content/drive/MyDrive/IARTS/dataset/crops/crops_images_yolo_original


In [25]:
# %pip install -q lime scikit-image  # se serve su Colab

import numpy as np, cv2, os
from ultralytics import YOLO
from lime.lime_image import LimeImageExplainer
from skimage.segmentation import slic, mark_boundaries
import matplotlib.pyplot as plt
import torch

# Carico i pesi già trovati dal notebook
device = 0 if torch.cuda.is_available() else "cpu"
model = YOLO(best_weights)
print("YOLO caricato da:", best_weights)

def plate_score_from_yolo(img_rgb):
    # restituisce uno score [0..1] "quanto è targa"
    if img_rgb.dtype != np.uint8:
        arr = np.clip(img_rgb * 255.0, 0, 255).astype(np.uint8)
    else:
        arr = img_rgb
    img_bgr = cv2.cvtColor(arr, cv2.COLOR_RGB2BGR)
    res = model.predict(img_bgr, imgsz=640, conf=0.001, iou=0.7, verbose=False, device=device)
    if not res or res[0] is None or res[0].boxes is None or len(res[0].boxes) == 0:
        return 0.0
    confs = res[0].boxes.conf.detach().cpu().numpy().astype(float)
    return float(np.clip(confs.max(), 0.0, 1.0))

def predict_proba_for_lime(batch_rgb):
    # LIME richiede probabilità per classi [non_targa, targa]
    probs = []
    for img in batch_rgb:
        s = plate_score_from_yolo(img)
        probs.append([1.0 - s, s])
    return np.array(probs, dtype=float)

explainer = LimeImageExplainer(random_state=0)

def explain_crop_with_lime(crop_path, num_features=10, num_samples=1000, n_segments=200):
    img_bgr = cv2.imread(crop_path)
    if img_bgr is None:
        print("Immagine non trovata:", crop_path); return None
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    def seg_fn(x):
        if x.dtype != np.uint8:
            x = np.clip(x*255.0, 0, 255).astype(np.uint8)
        return slic(x, n_segments=n_segments, compactness=10, sigma=1, start_label=1)

    exp = explainer.explain_instance(
        img_rgb,
        classifier_fn=predict_proba_for_lime,
        top_labels=2,
        hide_color=None,
        num_samples=num_samples,
        segmentation_fn=seg_fn
    )

    temp, mask = exp.get_image_and_mask(label=1, positive_only=True, num_features=num_features, hide_rest=False)
    return img_rgb, temp, mask


YOLO caricato da: /content/drive/MyDrive/IARTS/results/yolo_train/yolo_fuzzy_a0.1_b0.2_c0.6_d0.8/weights/best.pt


In [49]:
LIME_DIR = f"{RESULTS_DIR}/lime_{best_tag}"
CROPS_DIR_LIME = "/content/drive/MyDrive/IARTS/dataset/crops/crops_images_yolo_original"
os.makedirs(LIME_DIR, exist_ok=True)
exts = (".jpg",".jpeg",".png",".bmp",".tif",".tiff")
crop_list = sorted([p for p in os.listdir(CROPS_DIR_LIME) if p.lower().endswith(exts)])

print("Trovati crop:", len(crop_list))
for i, name in enumerate(crop_list, 1):
    p = os.path.join(CROPS_DIR_LIME, name)
    outname = os.path.splitext(name)[0] + "_lime.png"
    out_path = os.path.join(LIME_DIR, outname)

    res = explain_crop_with_lime(p, num_features=10, num_samples=1000, n_segments=200)
    if res is None:
        continue
    img_rgb, temp, mask = res

    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1); plt.imshow(img_rgb); plt.axis('off'); plt.title("Crop")
    plt.subplot(1,2,2); plt.imshow(mark_boundaries(temp.astype(np.uint8), mask)); plt.axis('off'); plt.title("LIME")
    plt.tight_layout(); plt.savefig(out_path, dpi=150); plt.close()

    if i % 20 == 0:
        print(f"{i}/{len(crop_list)}...")

print("Fatto. Output in:", LIME_DIR)


Trovati crop: 20


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

20/20...
Fatto. Output in: /content/drive/MyDrive/IARTS/results/lime_images_yolo_original


In [47]:
# OCR su TUTTE le cartelle: /content/drive/MyDrive/IARTS/dataset/crops/crops_yolo_*
# Output: un CSV per cartella + un CSV cumulativo in results/ocr_fastplate

import os, glob, json
import pandas as pd
from fast_plate_ocr import LicensePlateRecognizer

CROPS_ROOT = "/content/drive/MyDrive/IARTS/dataset/crops"
PATTERN    = "crops_yolo_*"
OUT_DIR    = "/content/drive/MyDrive/IARTS/results/ocr_fastplate"
os.makedirs(OUT_DIR, exist_ok=True)

m = LicensePlateRecognizer("cct-xs-v1-global-model")

def parse_result(res):
    """Rende robusto l'output: accetta dict/list/str e restituisce (plate, conf, raw)."""
    plate, conf, raw = None, None, res
    if isinstance(res, dict):
        for k in ["plate","text","license","lp","code","pred"]:
            if k in res:
                plate = res[k]; break
        for k in ["confidence","conf","score","prob","probability"]:
            if k in res:
                conf = res[k]; break
        raw = json.dumps(res, ensure_ascii=False)
    elif isinstance(res, (list, tuple)) and len(res) > 0:
        plate = res[0]
        try: conf = float(res[1]) if len(res) > 1 else None
        except: pass
        raw = json.dumps(res, ensure_ascii=False)
    else:
        plate = str(res)
    return plate, conf, raw

all_rows = []

folders = sorted(glob.glob(os.path.join(CROPS_ROOT, PATTERN)))
print("Cartelle trovate:", len(folders))
for crops_dir in folders:
    exts = (".jpg",".jpeg",".png",".bmp",".tif",".tiff")
    imgs = sorted([p for p in glob.glob(os.path.join(crops_dir, "*")) if p.lower().endswith(exts)])
    tag  = os.path.basename(crops_dir)
    out_csv = os.path.join(OUT_DIR, f"fastplate_{tag}.csv")

    print(f"→ {tag}: {len(imgs)} immagini")
    rows = []
    for i, p in enumerate(imgs, 1):
        try:
            res = m.run(p)
            plate, conf, raw = parse_result(res)
            rows.append({"set": tag, "crop_path": p, "plate": plate, "conf": conf, "raw": raw})
        except Exception as e:
            rows.append({"set": tag, "crop_path": p, "plate": None, "conf": None, "raw": f"ERROR: {e}"})
        if i % 50 == 0:
            print(f"   {i}/{len(imgs)}")

    pd.DataFrame(rows).to_csv(out_csv, index=False)
    print("   salvato:", out_csv)
    all_rows.extend(rows)

# CSV cumulativo
all_csv = os.path.join(OUT_DIR, "fastplate_ALL.csv")
pd.DataFrame(all_rows).to_csv(all_csv, index=False)
print("✅ Finito. CSV cumulativo:", all_csv)


Cartelle trovate: 8
→ crops_yolo_fuzzy_a0.1_b0.2_c0.6_d0.8: 22 immagini
   salvato: /content/drive/MyDrive/IARTS/results/ocr_fastplate/fastplate_crops_yolo_fuzzy_a0.1_b0.2_c0.6_d0.8.csv
→ crops_yolo_fuzzy_a0.2_b0.4_c0.7_d0.9: 23 immagini
   salvato: /content/drive/MyDrive/IARTS/results/ocr_fastplate/fastplate_crops_yolo_fuzzy_a0.2_b0.4_c0.7_d0.9.csv
→ crops_yolo_high_20: 23 immagini
   salvato: /content/drive/MyDrive/IARTS/results/ocr_fastplate/fastplate_crops_yolo_high_20.csv
→ crops_yolo_high_30: 20 immagini
   salvato: /content/drive/MyDrive/IARTS/results/ocr_fastplate/fastplate_crops_yolo_high_30.csv
→ crops_yolo_high_40: 23 immagini
   salvato: /content/drive/MyDrive/IARTS/results/ocr_fastplate/fastplate_crops_yolo_high_40.csv
→ crops_yolo_low_20: 34 immagini
   salvato: /content/drive/MyDrive/IARTS/results/ocr_fastplate/fastplate_crops_yolo_low_20.csv
→ crops_yolo_low_30: 20 immagini
   salvato: /content/drive/MyDrive/IARTS/results/ocr_fastplate/fastplate_crops_yolo_low_30.csv
→ 

In [48]:
# OCR SOLO nella cartella: /content/drive/MyDrive/IARTS/dataset/images
# Output: un CSV nella cartella results/ocr_fastplate + CSV cumulativo (identico)

import os, glob, json
import pandas as pd
from fast_plate_ocr import LicensePlateRecognizer

IMAGES_DIR = "/content/drive/MyDrive/IARTS/dataset/crops/crops_images_yolo_original"
OUT_DIR    = "/content/drive/MyDrive/IARTS/results/ocr_fastplate_original_images"
os.makedirs(OUT_DIR, exist_ok=True)

m = LicensePlateRecognizer("cct-xs-v1-global-model")

def parse_result(res):
    """Rende robusto l'output: accetta dict/list/str e restituisce (plate, conf, raw)."""
    plate, conf, raw = None, None, res
    if isinstance(res, dict):
        for k in ["plate","text","license","lp","code","pred"]:
            if k in res:
                plate = res[k]; break
        for k in ["confidence","conf","score","prob","probability"]:
            if k in res:
                conf = res[k]; break
        raw = json.dumps(res, ensure_ascii=False)
    elif isinstance(res, (list, tuple)) and len(res) > 0:
        plate = res[0]
        try: conf = float(res[1]) if len(res) > 1 else None
        except: pass
        raw = json.dumps(res, ensure_ascii=False)
    else:
        plate = str(res)
    return plate, conf, raw

exts = (".jpg",".jpeg",".png",".bmp",".tif",".tiff")
imgs = sorted([p for p in glob.glob(os.path.join(IMAGES_DIR, "*")) if p.lower().endswith(exts)])
tag  = os.path.basename(IMAGES_DIR.rstrip("/"))
out_csv = os.path.join(OUT_DIR, f"fastplate_{tag}.csv")

print(f"Cartella '{tag}': {len(imgs)} immagini")
rows = []
for i, p in enumerate(imgs, 1):
    try:
        res = m.run(p)
        plate, conf, raw = parse_result(res)
        rows.append({"set": tag, "crop_path": p, "plate": plate, "conf": conf, "raw": raw})
    except Exception as e:
        rows.append({"set": tag, "crop_path": p, "plate": None, "conf": None, "raw": f"ERROR: {e}"})
    if i % 50 == 0:
        print(f"   {i}/{len(imgs)}")

# CSV singolo
pd.DataFrame(rows).to_csv(out_csv, index=False)
print("   salvato:", out_csv)

# CSV cumulativo (uguale perché c'è solo una cartella)
all_csv = os.path.join(OUT_DIR, "fastplate_ALL_original.csv")
pd.DataFrame(rows).to_csv(all_csv, index=False)
print("✅ Finito. CSV cumulativo:", all_csv)


Cartella 'crops_images_yolo_original': 20 immagini
   salvato: /content/drive/MyDrive/IARTS/results/ocr_fastplate_original_images/fastplate_crops_images_yolo_original.csv
✅ Finito. CSV cumulativo: /content/drive/MyDrive/IARTS/results/ocr_fastplate_original_images/fastplate_ALL_original.csv
